# Tweets sentiment analysis using Pyspark
## The sentiment scores are assigned using Bing Liu opinion lexicon

In [174]:
import tweepy
from tweepy import OAuthHandler
import csv

In [175]:
### API to query for #hospital and get atleast 1000 tweets

import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

access_key = '1314619141517701126-XLK51FotjzkViLWCavDAK3bQScy5Gp'
access_secret = 'tR25RRQoIj6h8T4eGSU0EyJTN5CJ97MEccEZkqzydAS5p'
consumer_key = 'Y5PySZEtOGePMEmCy2LgfDe5P'
consumer_secret = 'RJ1OttT9CiFPWwdQfhc2Uzscjcpg5pEdcJotEgAxZlb6aK1wjU'

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_key, access_secret)
#twitterStream=Stream(auth,listener())
#twitterStream.filter(track=["#hospital"])
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

query = '#hospital'
max_tweets = 2500
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]


In [176]:
###Checking number of tweets
len(searched_tweets)

2500

In [177]:
hospital_tweets = []
for tweets in searched_tweets:
    hospital_tweets.append(tweets.text)

In [178]:
from pyspark.sql import Row
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

rdd = sc.parallelize(hospital_tweets)
row_rdd = rdd.map(lambda x: Row(x))

#### Tweets dataframe
spark_df=sqlContext.createDataFrame(row_rdd,['Tweets'])

In [179]:
######### Removing punctuation, special characters and numbers

from pyspark.sql import SparkSession, functions as F 
spark_df = spark_df.withColumn("Tweet_content", F.regexp_replace(spark_df.Tweets, r"[^a-zA-Z\. ]",""))

In [180]:
spark_df.show(5)

+--------------------+--------------------+
|              Tweets|       Tweet_content|
+--------------------+--------------------+
|RT @SeufertRecht:...|RT SeufertRecht S...|
|RT @SeufertRecht:...|RT SeufertRecht S...|
|#Corona #COVID19 ...|Corona COVID COVI...|
|Durga Puja
#durga...|Durga Pujadurgapu...|
|Breaking News: 24...|Breaking News Oct...|
+--------------------+--------------------+
only showing top 5 rows



In [181]:
#!pip install langdetect

In [182]:
### Creating user-defined function to check the language
from langdetect import detect
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import StringType

def check_lang(data_str):
    predict_lang = detect(data_str)
    if predict_lang == 'en':
        language = predict_lang
    else:
        language = 'NA'
    return language    
check_lang_udf = udf(check_lang, StringType())

In [183]:
###Only kept the english language for accurate sentiment score detection

spark_df = spark_df.withColumn("Language", check_lang_udf(spark_df['Tweet_content']))
spark_df = spark_df.filter(spark_df.Language == 'en')

In [184]:
spark_df.count()   ##### to ensure atleast 1000 tweets

1766

In [185]:
spark_df.show(5)

+--------------------+--------------------+--------+
|              Tweets|       Tweet_content|Language|
+--------------------+--------------------+--------+
|RT @SeufertRecht:...|RT SeufertRecht S...|      en|
|RT @SeufertRecht:...|RT SeufertRecht S...|      en|
|#Corona #COVID19 ...|Corona COVID COVI...|      en|
|Breaking News: 24...|Breaking News Oct...|      en|
|41 million adults...| million adults i...|      en|
+--------------------+--------------------+--------+
only showing top 5 rows



In [186]:
from pyspark.ml.feature import RegexTokenizer

In [187]:
# the tokenizer object
tokenizer = RegexTokenizer().setInputCol('Tweet_content').setOutputCol('words')
#### Regex tokenizer first converts the words to lower space and then splits by white-spaces

In [188]:
hospital_df = tokenizer.transform(spark_df)#.select('tweets_new','words')

In [189]:
###### checking dataset into tall format

hospital_df.select('Tweet_content', F.explode('words').alias('word')).show(5)

+--------------------+------------+
|       Tweet_content|        word|
+--------------------+------------+
|RT SeufertRecht S...|          rt|
|RT SeufertRecht S...|seufertrecht|
|RT SeufertRecht S...|     seufert|
|RT SeufertRecht S...|     advises|
|RT SeufertRecht S...|       peine|
+--------------------+------------+
only showing top 5 rows



In [190]:
######### Downlaod Bing Liu opinion lexicon files 

### negative words
neg=sc.textFile("C:\\Users\\rahul\\Documents\\big_data\\opinion-lexicon-English\\opinion-lexicon-English\\negative-words.txt")
temp_var = neg.map(lambda k: k.split("\\t"))
neg_df = temp_var.toDF()
neg_df= neg_df.withColumn('sentiment',lit(-1))

### positive words
pos=sc.textFile("C:\\Users\\rahul\\Documents\\big_data\\opinion-lexicon-English\\opinion-lexicon-English\\positive-words.txt")
temp_var_pos = pos.map(lambda k: k.split("\\t"))
pos_df = temp_var_pos.toDF()
pos_df= pos_df.withColumn('sentiment',lit(1))

#### concat both
words_concat = pos_df.union(neg_df)
columns = ['word','sentiment']
words = words_concat.toDF(*columns)
words.show(5)

+---------+---------+
|     word|sentiment|
+---------+---------+
|       a+|        1|
|   abound|        1|
|  abounds|        1|
|abundance|        1|
| abundant|        1|
+---------+---------+
only showing top 5 rows



In [191]:
####### joining hospital tweets with Liu Bing words to get sentiment of each word

join_df = hospital_df.\
    select('Tweet_content', F.explode('words').alias('word')).\
    join(words, 'word')
join_df.show(10)

+------------+--------------------+---------+
|        word|       Tweet_content|sentiment|
+------------+--------------------+---------+
|inflammatory|RT frhossain Infl...|       -1|
|    randomly|RT kellyarchives ...|       -1|
|    randomly|RT kellyarchives ...|       -1|
|    randomly|octors randomly r...|       -1|
|    positive|Does the hospital...|        1|
|    positive|Name  Ganta Guru ...|        1|
|    positive|RT SaraECores My ...|        1|
|    positive|My dad is a surge...|        1|
|  productive|Sthule syabuka nj...|        1|
|      speedy|Ranveer singh has...|        1|
+------------+--------------------+---------+
only showing top 10 rows



In [192]:
####### Grouping sentiments by tweets and using sum of the sentiments as score
score_df = join_df.\
    groupBy('Tweet_content').\
    agg(F.sum('sentiment').alias('Sentiment_score'))

score_df.sort(col('Sentiment_score').desc()).show(5)

+--------------------+---------------+
|       Tweet_content|Sentiment_score|
+--------------------+---------------+
|RT HumanityFirstU...|             48|
|RT CTVNews An yea...|             24|
|RT DrNinaRadcliff...|              9|
|RT paulbami Dwell...|              9|
|RT CleveClinicFL ...|              8|
+--------------------+---------------+
only showing top 5 rows



In [193]:
########## Using original tweets instead of cleaned tweets in results
final_tweets = hospital_df.\
    select('Tweets','Tweet_content').\
    join(score_df, 'Tweet_content')

In [194]:
final_tweets.select('Tweets','Sentiment_score').distinct().sort(col('Sentiment_score').desc()).show(5,False)

+------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Tweets                                                                                                                                          |Sentiment_score|
+------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|RT @HumanityFirstUK: 🏥 Work is well underway for the Brand New #Hospital being built in Yopougon, south of Côte d’Ivoire #IvoryCoast

You c…   |48             |
|RT @CTVNews: An 81-year-old COVID-19 patient in Australia was discharged after 214 days in hospital. #ctvnews #coronavirus #pandemic #covid…    |24             |
|RT @paulbami: Dwellers around #NEPA, #SIJUADE, #ESO, #HOSPITAL ROAD, LETS MEET @NEPA to clean that road by 7:00am
What do you think?
@iam_h…    |9              |
|RT @DrNinaRadcliff: Ha

In [173]:
write_df=final_tweets.select('Tweets','Sentiment_score').distinct().sort(col('Sentiment_score').desc())   ### desired output csv asked

###### Saving in csv
write_df.coalesce(1).write.csv('final_sentiments',header = 'true')